In [11]:
import openai
from dotenv import load_dotenv, find_dotenv
import os
import sys

sys.path.append("../..")


_ = load_dotenv(find_dotenv())  # read local .env file

HUGGINGFACEHUB_API_TOKEN = os.environ["HUGGINGFACEHUB_API_TOKEN"]

openai.api_key = os.environ["OPENAI_API_KEY"]

In [17]:
import langchain

langchain.debug = False

In [13]:
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

embeddings = HuggingFaceInferenceAPIEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    api_key=HUGGINGFACEHUB_API_TOKEN,
)

In [4]:
mdc = """

Welcome to MDC Media, where expertise meets innovation. As a dynamic technology company based in Hanoi City, we specialize in mobile development and web design, catering to individuals, companies, and startups. Our dedicated team, guided by the mantra "One app at a time," is committed to transforming your ideas into cutting-edge solutions.

At MDC, our specialized teams seamlessly blend strategy, technology, data science, and design to tackle the most intricate business challenges. Whether you're a high-growth company or a startup with bold ideas, we lead the product design and development process with precision and enthusiasm.

- *Web Application:* MDC SOFTWARE is committed to building the highest quality applications.
- *Artificial Intelligence:* Delve into the future with our expertise in AI, IoT, and big data analysis.
- *UX Design:* Our professional design team crafts intuitive and user-friendly applications.
- *Streaming Solution:* We develop audio/video streaming systems capable of supporting millions of concurrent users.
- *Product Marketing:* Harness the power of modern social media to reach millions of users cost-effectively.
- *Payment Integration:* Seamlessly integrate common payment gateways like PayPal, VISA/MASTER.
- *Blockchain:* Benefit from our extensive experience in blockchain technology, renowned in the region.
- *Technical Advice:* We provide comprehensive technical advice, support, and solutions for your IT infrastructure.

Explore our diverse portfolio, including projects like IPFM for Digital Audio Broadcasting, eKYC for Digital Customer Onboarding, and innovative mobile applications such as iVPN, iMatch, and Advanced IPTV Player. Our experiences also extend to Augmented Reality, exemplified by projects like Can Knockdown - AR and Living In The Battles.

Behind every successful product is a dedicated team. Meet our professionals who excel in idealizing, analyzing, designing, and developing products. We believe in creating better user experiences with a systematic and rigorous approach.

Are you passionate about mobile development and web design? MDC Media is actively recruiting frontend React Native developers and freshers. Join us on our journey to create groundbreaking projects, and be part of a team that values learning, growth, and innovation.


Feel inspired to connect with us? Whether you're seeking our services or considering a career with MDC, we're here for you. Complete the form on our Contact page, and our team will reach out promptly.

MDC VIETNAM MEDIA AND TECHNOLOGY COMPANY LIMITED
15th Floor, Viet A Building, No. 9 Duy Tan, Cau Giay, Hanoi
Contact: 0868.733.900
Email: hello@mdcsoftware.com.vn

Stay updated with MDC Media on social media for the latest news, projects, and industry insights.

At MDC Media, we're not just building applications; we're building experiences. Join us in shaping the future—one app at a time. Copyright © 2021 MDC MEDIA CO.,LTD. All Rights Reserved.
"""
summary = mdc.split("\n\n")[1:]

In [14]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS

db = await FAISS.afrom_texts(summary, embeddings)
query = "Name some of the projects MDC Software has worked on"
docs = await db.asimilarity_search(query)

print(docs)

[Document(page_content="At MDC, our specialized teams seamlessly blend strategy, technology, data science, and design to tackle the most intricate business challenges. Whether you're a high-growth company or a startup with bold ideas, we lead the product design and development process with precision and enthusiasm."), Document(page_content='- *Web Application:* MDC SOFTWARE is committed to building the highest quality applications.\n- *Artificial Intelligence:* Delve into the future with our expertise in AI, IoT, and big data analysis.\n- *UX Design:* Our professional design team crafts intuitive and user-friendly applications.\n- *Streaming Solution:* We develop audio/video streaming systems capable of supporting millions of concurrent users.\n- *Product Marketing:* Harness the power of modern social media to reach millions of users cost-effectively.\n- *Payment Integration:* Seamlessly integrate common payment gateways like PayPal, VISA/MASTER.\n- *Blockchain:* Benefit from our exten

In [36]:
from langchain.chat_models import ChatOpenAI
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
    RunnableParallel,
)
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

retriever = db.as_retriever()

template = """You're helpful assistant, please answer the question based only on the following context:
{context}
Your response based on user's question language.
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(
    model_name="gpt-3.5-turbo-1106",
    temperature=0,
    callbacks=[StreamingStdOutCallbackHandler()],
    streaming=True,
)
chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

In [38]:
chain.invoke("Văn phòng của MDC ở quận nào?")

Trụ sở chính của MDC nằm tại tòa nhà Viet A, số 9 Đuy Tân, quận Cầu Giấy, Hà Nội.

'Trụ sở chính của MDC nằm tại tòa nhà Viet A, số 9 Đuy Tân, quận Cầu Giấy, Hà Nội.'